In [5]:
! pip install python-dotenv ipython-cypher neo4j py2neo yfiles_jupyter_graphs graphdatascience pandas --quiet

In [6]:
import os
%load_ext dotenv
%dotenv

PATIENT_DATA_PATH=os.getenv("PATIENT_DATA_PATH")
NEO4J_HOST=os.getenv("NEO4J_HOST")
NEO4J_USER=os.getenv("NEO4J_USER")
NEO4J_PASSWORD=os.getenv("NEO4J_PASSWORD")

In [7]:
from neo4j import GraphDatabase

uri = "bolt://{host}:7687".format(host=NEO4J_HOST)

with GraphDatabase.driver(uri, auth=(NEO4J_USER, NEO4J_PASSWORD)) as driver: 
    driver.verify_connectivity() 
    session = driver.session(database='')

<ipython-input-7-32dba96e4a69>:6: ExperimentalWarning: The configuration may change in the future.
  driver.verify_connectivity()


In [8]:
def cypher_run(cypher):
    result = session.run(cypher)
    return result.to_df()

def cypher_run_g(cypher):
    result = session.run(cypher)
    return result.graph()

# Import Patient Data to Neo4j

Queries from: https://github.com/Neo4jSolutions/patient-journey-model/blob/master/ingest/config.yml   (Slightly changed)

## Payers

In [4]:
q = """
LOAD CSV WITH HEADERS FROM 'http://fileserver:4858/patient_data/csv/payers.csv' AS row
MERGE (p:Payer {id:row.Id})
          SET p.name=row.NAME,
            p.address=row.ADDRESS,
            p.city=row.CITY,
            p.zip=row.ZIP,
            p.state=row.STATE_HEADQUARTERED
"""
cypher_run(q)

<ipython-input-3-e79678bb85b2>:3: ExperimentalWarning: pandas support is experimental and might be changed or removed in future versions
  return result.to_df()


""


In [7]:
q = """ MATCH (p:Payer) return p.id LIMIT 5; """
cypher_run(q)

<ipython-input-3-e79678bb85b2>:3: ExperimentalWarning: pandas support is experimental and might be changed or removed in future versions
  return result.to_df()


,p.id
0,a735bf55-83e9-331a-899d-a82a60b9f60c
1,df166300-5a78-3502-a46a-832842197811
2,d18ef2e6-ef40-324c-be54-34a5ee865625
3,26aab0cd-6aba-3e1b-ac5b-05c8867e762c
4,b046940f-1664-3047-bca7-dfa76be352a4


 ## Encounters

In [12]:
q = """
    LOAD CSV WITH HEADERS FROM 'http://fileserver:4858/patient_data/csv/encounters.csv' AS row
    MERGE (e:Encounter {id:row.Id})
      SET e.code=row.CODE,
        e.description=row.DESCRIPTION,
        e.class=row.ENCOUNTERCLASS,
        e.date=datetime(row.START),
        e.baseCost=toFloat(row.BASE_ENCOUNTER_COST),
        e.claimCost=toFloat(row.TOTAL_CLAIM_COST),
        e.coveredAmount=toFloat(row.PAYER_COVERAGE),
        e.isEnd=false
    FOREACH (ignore in CASE WHEN row.STOP IS NOT NULL AND row.STOP <> '' THEN [1] ELSE [] END |
      SET e.end=datetime(row.STOP)
    )
    MERGE (p:Patient {id:row.PATIENT})
    MERGE (p)-[:HAS_ENCOUNTER]->(e)
    FOREACH (ignore in CASE WHEN row.PROVIDER IS NOT NULL AND row.PROVIDER <> '' THEN [1] ELSE [] END |
      MERGE (o:Organization {id: row.PROVIDER})
      MERGE (e)-[:HAS_PROVIDER]->(o))
    WITH e,row
    MATCH (pa:Payer {id:row.PAYER})
    MERGE (e)-[:HAS_PAYER]->(pa)
"""
cypher_run(q)

<ipython-input-3-e79678bb85b2>:3: ExperimentalWarning: pandas support is experimental and might be changed or removed in future versions
  return result.to_df()


""


In [13]:
q = """ MATCH (e:Encounter) return e.id LIMIT 5; """
cypher_run(q)

<ipython-input-3-e79678bb85b2>:3: ExperimentalWarning: pandas support is experimental and might be changed or removed in future versions
  return result.to_df()


,e.id
0,c24f2498-f156-217d-c45b-23665b0650c6
1,5a8860b9-d95e-e4cf-e2bc-bf751eb6a847
2,dc8a523c-b180-8631-395a-d3208ae6242d
3,5e051e99-229b-ba9a-5298-8dcba6bd91a7
4,487df578-2eb5-cd17-39c6-3201744ed1dd


## Providers

In [9]:
q = """
    LOAD CSV WITH HEADERS FROM 'http://fileserver:4858/patient_data/csv/providers.csv' AS row
    MERGE (p:Provider {id: row.Id})
    SET p.name=row.NAME,
      p.speciality=row.SPECIALITY
    MERGE (o:Organization {id: row.ORGANIZATION})
    MERGE (p)-[:BELONGS_TO]->(o)
    MERGE (a:Address {address: row.ADDRESS})
    SET a.location = point({latitude:toFloat(row.LAT), longitude:toFloat(row.LON)})
    MERGE (p)-[:HAS_ADDRESS]->(a)
"""
cypher_run(q)

<ipython-input-8-e79678bb85b2>:3: ExperimentalWarning: pandas support is experimental and might be changed or removed in future versions
  return result.to_df()


""


In [10]:
q = """ MATCH (p:Provider) return p.id LIMIT 5; """
cypher_run(q)

<ipython-input-8-e79678bb85b2>:3: ExperimentalWarning: pandas support is experimental and might be changed or removed in future versions
  return result.to_df()


,p.id
0,02290f60-0f0a-338c-ab5e-4c95aa8f6597
1,75fda570-a6bb-3b7e-8491-173b1830b3e6
2,ae2fed4d-3177-3f5b-83e5-8def64e2a2b9
3,86d0f508-444a-3d80-baba-53cb0ad89c64
4,5c894baf-eed5-3715-80c2-2fa5c8b8cf22


## Patients

In [11]:
q = """
        LOAD CSV WITH HEADERS FROM 'http://fileserver:4858/patient_data/csv/patients.csv' AS row
        MERGE (p:Patient {id:row.Id})
          SET
            p.birthDate=datetime(row.BIRTHDATE),
            p.deathDate=row.DEATHDATE,
            p.firstName=row.FIRST,
            p.lastName=row.LAST,
            p.SSN=row.SSN,
            p.marital=row.MARITAL,
            p.gender=row.GENDER,
            p.race=row.RACE,
            p.ethnicity=row.ETHNICITY,
            p.city=row.CITY
        MERGE (a:Address {address: row.ADDRESS})
        SET a.location = point({latitude:toFloat(row.LAT), longitude:toFloat(row.LON)})
        MERGE (p)-[:HAS_ADDRESS]->(a)
"""
cypher_run(q)

<ipython-input-8-e79678bb85b2>:3: ExperimentalWarning: pandas support is experimental and might be changed or removed in future versions
  return result.to_df()


""


In [12]:
q = """ MATCH (p:Patient) return p.id LIMIT 5; """
cypher_run(q)

<ipython-input-8-e79678bb85b2>:3: ExperimentalWarning: pandas support is experimental and might be changed or removed in future versions
  return result.to_df()


,p.id
0,358cb878-9fb6-e36f-45b8-34c3bc7b1489
1,d46afbe3-e7b8-c047-6027-8d6d0b59be57
2,879355dc-489a-fcbb-3466-c5483fb18c0e
3,33873583-872d-0765-5501-c89a85c5580d
4,18d1ab89-99d4-0733-3fde-8d2f272e17de


## Payer Transitions

In [13]:
q = """
    LOAD CSV WITH HEADERS FROM 'http://fileserver:4858/patient_data/csv/payer_transitions.csv' AS row
    MATCH (p:Patient {id:row.PATIENT})
    MATCH (payer:Payer {id:row.PAYER})
    CREATE (p)-[s:INSURANCE_START]->(payer)
    SET s.year=toInteger(row.START_YEAR)
    CREATE (p)-[e:INSURANCE_END]->(payer)
    SET e.year=toInteger(row.END_YEAR)
"""
cypher_run(q)

<ipython-input-8-e79678bb85b2>:3: ExperimentalWarning: pandas support is experimental and might be changed or removed in future versions
  return result.to_df()


""


In [14]:
q = """ MATCH (p:Payer) return p.id LIMIT 5; """
cypher_run(q)

<ipython-input-8-e79678bb85b2>:3: ExperimentalWarning: pandas support is experimental and might be changed or removed in future versions
  return result.to_df()


,p.id
0,a735bf55-83e9-331a-899d-a82a60b9f60c
1,df166300-5a78-3502-a46a-832842197811
2,d18ef2e6-ef40-324c-be54-34a5ee865625
3,26aab0cd-6aba-3e1b-ac5b-05c8867e762c
4,b046940f-1664-3047-bca7-dfa76be352a4


## Allergies

In [16]:
q = """
        LOAD CSV WITH HEADERS FROM 'http://fileserver:4858/patient_data/csv/allergies.csv' AS row
          MATCH (p:Patient {id:row.PATIENT})
          MERGE (a:Allergy {code:row.CODE})
          SET a.description=row.DESCRIPTION
          MERGE (as:Encounter {id:row.ENCOUNTER, isEnd: false})
          ON CREATE
            SET as.date=datetime(row.START), as.code=row.CODE
          ON MATCH
            SET as.code=row.CODE
          MERGE (p)-[:HAS_ENCOUNTER]->(as)
          MERGE (as)-[:HAS_ALLERGY]->(a)
          WITH p,a,as,row
          WHERE row.STOP IS NOT NULL and row.STOP <> ''
          MERGE (ae:Encounter {id:row.ENCOUNTER, date:datetime(row.STOP)})
          SET ae.code=row.CODE, ae.isEnd=true
          MERGE (p)-[:HAS_ENCOUNTER]->(ae)
          MERGE (ae)-[:HAS_ALLERGY]->(a)
          MERGE (as)-[:HAS_END]->(ae)
"""
cypher_run(q)

<ipython-input-8-e79678bb85b2>:3: ExperimentalWarning: pandas support is experimental and might be changed or removed in future versions
  return result.to_df()


""


In [17]:
q = """ MATCH (a:Allergy) return a.code LIMIT 5; """
cypher_run(q)

<ipython-input-8-e79678bb85b2>:3: ExperimentalWarning: pandas support is experimental and might be changed or removed in future versions
  return result.to_df()


,a.code
0,419199007
1,442571000124108
2,264287008
3,412071004
4,29046


## Conditions

In [18]:
q = """
    LOAD CSV WITH HEADERS FROM 'http://fileserver:4858/patient_data/csv/conditions.csv' AS row
        MATCH (p:Patient {id:row.PATIENT})
        MERGE (c:Condition {code:row.CODE})
        SET c.description=row.DESCRIPTION
        MERGE (cs:Encounter {id:row.ENCOUNTER, isEnd: false})
        ON CREATE
          SET cs.date=datetime(row.START), cs.code=row.CODE
        ON MATCH
          SET cs.code=row.CODE
        MERGE (p)-[:HAS_ENCOUNTER]->(cs)
        MERGE (cs)-[:HAS_CONDITION]->(c)
        WITH p,c,cs,row
        WHERE row.STOP IS NOT NULL and row.STOP <> ''
        MERGE (ce:Encounter {id:row.ENCOUNTER, date:datetime(row.STOP)})
        SET ce.code=row.CODE, ce.isEnd=true
        MERGE (p)-[:HAS_ENCOUNTER]->(ce)
        MERGE (ce)-[:HAS_CONDITION]->(c)
        MERGE (cs)-[:HAS_END]->(ce)
"""
cypher_run(q)

<ipython-input-8-e79678bb85b2>:3: ExperimentalWarning: pandas support is experimental and might be changed or removed in future versions
  return result.to_df()


""


In [19]:
q = """ MATCH (c:Condition) return c.code LIMIT 5; """
cypher_run(q)

<ipython-input-8-e79678bb85b2>:3: ExperimentalWarning: pandas support is experimental and might be changed or removed in future versions
  return result.to_df()


,c.code
0,314529007
1,75498004
2,40055000
3,224299000
4,160903007


 ## Medications

In [20]:
q = """
        LOAD CSV WITH HEADERS FROM 'http://fileserver:4858/patient_data/csv/medications.csv' AS row
           MERGE (p:Patient {id:row.PATIENT})
          MERGE (d:Drug {code:row.CODE})
          SET d.description=row.DESCRIPTION
          MERGE (ps:Encounter {id:row.ENCOUNTER, isEnd: false})
          ON CREATE
            SET ps.code=row.CODE, ps.date=datetime(row.START)
          ON MATCH
            SET ps.code=row.CODE
          MERGE (p)-[:HAS_ENCOUNTER]->(ps)
          MERGE (ps)-[:HAS_DRUG]->(d)
          WITH p,d,ps,row
          WHERE row.STOP IS NOT NULL and row.STOP <> ''
          CREATE (pe:Encounter {id:row.ENCOUNTER, date:datetime(row.STOP)})
          SET pe.code=row.CODE, pe.isEnd=true
          MERGE (p)-[:HAS_ENCOUNTER]->(pe)
          MERGE (pe)-[:HAS_DRUG]->(d)
          MERGE (ps)-[:HAS_END]->(pe)
"""
cypher_run(q)

<ipython-input-8-e79678bb85b2>:3: ExperimentalWarning: pandas support is experimental and might be changed or removed in future versions
  return result.to_df()


""


In [21]:
q = """ MATCH (d:Drug) return d.code LIMIT 5; """
cypher_run(q)

<ipython-input-8-e79678bb85b2>:3: ExperimentalWarning: pandas support is experimental and might be changed or removed in future versions
  return result.to_df()


,d.code
0,665078
1,1870230
2,857005
3,849574
4,834061


## Procedures

In [22]:
q = """
    LOAD CSV WITH HEADERS FROM 'http://fileserver:4858/patient_data/csv/procedures.csv' AS row
          MERGE (p:Patient {id:row.PATIENT})
          MERGE (r:Procedure {code:row.CODE})
          SET r.description=row.DESCRIPTION
          MERGE (pe:Encounter {id:row.ENCOUNTER, isEnd: false})
          ON CREATE
            SET pe.date=datetime(row.START), pe.code=row.CODE
          ON MATCH
            SET pe.code=row.CODE
          MERGE (p)-[:HAS_ENCOUNTER]->(pe)
          MERGE (pe)-[:HAS_PROCEDURE]->(r)
"""
cypher_run(q)

<ipython-input-8-e79678bb85b2>:3: ExperimentalWarning: pandas support is experimental and might be changed or removed in future versions
  return result.to_df()


""


In [23]:
q = """ MATCH (p:Procedure) return p.code LIMIT 5; """
cypher_run(q)

<ipython-input-8-e79678bb85b2>:3: ExperimentalWarning: pandas support is experimental and might be changed or removed in future versions
  return result.to_df()


,p.code
0,171207006
1,715252007
2,710841007
3,428211000124100
4,868187001


## Observations

In [29]:
# observations
q = """
    LOAD CSV WITH HEADERS FROM 'http://fileserver:4858/patient_data/csv/observations.csv' AS row
      WITH row
      WHERE row.ENCOUNTER IS NOT NULL and row.ENCOUNTER <> ''
      WITH row, row.CODE as code, CASE row.TYPE WHEN 'text' THEN row.VALUE ELSE toFloat(row.VALUE) END as value
      WITH row, apoc.map.fromPairs([[code, value]]) as attr
      MATCH (p:Patient {id:row.PATIENT})
      MATCH (oe:Encounter {id:row.ENCOUNTER, isEnd: false})
      MERGE (oe) -[:HAS_OBSERVATION]-> (o:Observation )
      SET o += attr
"""
cypher_run(q)

<ipython-input-8-e79678bb85b2>:3: ExperimentalWarning: pandas support is experimental and might be changed or removed in future versions
  return result.to_df()


""


In [30]:
q = """ MATCH (oe:Encounter)-[:HAS_OBSERVATION]-> (o:Observation ) return oe.id LIMIT 5; """
cypher_run(q)

<ipython-input-8-e79678bb85b2>:3: ExperimentalWarning: pandas support is experimental and might be changed or removed in future versions
  return result.to_df()


,oe.id
0,5e051e99-229b-ba9a-5298-8dcba6bd91a7
1,1600e625-65b1-2acb-8a9f-ef3eabbf7e44
2,9573e48f-20e0-aae6-47ca-4f60db51d50d
3,e5fed907-1d5a-8a47-2013-ab6e6770b07e
4,aca906bb-cdbc-de3f-2c85-cabc8f5139d3


## Care Plans

In [33]:
# care plans
q = """
    LOAD CSV WITH HEADERS FROM 'http://fileserver:4858/patient_data/csv/careplans.csv' AS row
    MATCH (p:Patient {id:row.PATIENT})
    MERGE (c:CarePlan {code:row.CODE})
    SET c.description=row.DESCRIPTION
    MERGE (cs:Encounter {id:row.ENCOUNTER, isEnd: false})
    ON CREATE
    SET cs.code=row.CODE, cs.date=datetime(row.START)
    ON MATCH
    SET cs.code=row.CODE
    MERGE (p)-[:HAS_ENCOUNTER]->(cs)
    MERGE (cs)-[:HAS_CARE_PLAN]->(c)
    WITH p,c,cs,row
    WHERE row.STOP IS NOT NULL and row.STOP <> ''
    CREATE (ce:Encounter {id:row.ENCOUNTER, date:datetime(row.STOP)})
    SET ce.code=row.CODE, ce.isEnd=true
    MERGE (p)-[:HAS_ENCOUNTER]->(ce)
    MERGE (ce)-[:HAS_CARE_PLAN]->(c)
    MERGE (cs)-[:HAS_END]->(ce)
"""
cypher_run(q)

<ipython-input-8-e79678bb85b2>:3: ExperimentalWarning: pandas support is experimental and might be changed or removed in future versions
  return result.to_df()


""


In [34]:
q = """ MATCH (e:Encounter)-[:HAS_CARE_PLAN]->(c:CarePlan) return c.code LIMIT 5; """
cypher_run(q)

<ipython-input-8-e79678bb85b2>:3: ExperimentalWarning: pandas support is experimental and might be changed or removed in future versions
  return result.to_df()


,c.code
0,384758001
1,384758001
2,384758001
3,384758001
4,384758001


## Organizations

In [36]:
# organizations
q = """
    LOAD CSV WITH HEADERS FROM 'http://fileserver:4858/patient_data/csv/organizations.csv' AS row        MATCH (o:Organization {id:row.Id})
              SET o.name=row.NAME
            MERGE (a:Address {address: row.ADDRESS})
              SET a.location = point({latitude:toFloat(row.LAT), longitude:toFloat(row.LON)})
            MERGE (o)-[:HAS_ADDRESS]->(a)
    """
cypher_run(q)

<ipython-input-8-e79678bb85b2>:3: ExperimentalWarning: pandas support is experimental and might be changed or removed in future versions
  return result.to_df()


""


In [37]:
q = """ MATCH (o:Organization)-[:HAS_ADDRESS]->(a:Address) return o.id,a.address LIMIT 5; """
cypher_run(q)

<ipython-input-8-e79678bb85b2>:3: ExperimentalWarning: pandas support is experimental and might be changed or removed in future versions
  return result.to_df()


,o.id,a.address
0,74ab949d-17ac-3309-83a0-13b4405c66aa,881 Main Street
1,faffaf6a-ee1a-3673-b0b0-421a9c249244,66 WASHINGTON ST
2,4112b8b1-59df-3255-a7ca-f42ee0a4cb2e,37 ROUTE 6A
3,e09d4c49-c2ef-3b0f-9a46-3719d9219306,60 HOSPITAL RD
4,ef5390b4-cba5-3d83-96db-af783a7adb43,360 MERRIMACK ST


In [56]:
q = """
  
    CREATE INDEX IF NOT EXISTS FOR (e:Encounter) ON (e.id);
    CREATE INDEX IF NOT EXISTS FOR (e:Encounter) ON (e.date);
    CREATE INDEX IF NOT EXISTS FOR (e:Encounter) ON (e.isEnd);
    CREATE INDEX IF NOT EXISTS FOR (p:Patient) ON (p.id);
    CREATE INDEX IF NOT EXISTS FOR (p:Provider) ON (p.id);
    CREATE INDEX IF NOT EXISTS FOR (p:Payer) ON (p.id);    
    CREATE INDEX IF NOT EXISTS FOR (o:Organization) ON (o.id);
    CREATE INDEX IF NOT EXISTS FOR (d:Drug) ON (d.code);
    CREATE INDEX IF NOT EXISTS FOR (c:Condition) ON (c.code);
    CREATE INDEX IF NOT EXISTS FOR (c:CarePlan) ON (c.id);
    CREATE INDEX IF NOT EXISTS FOR (s:Speciality) ON (s.name);
    CREATE INDEX IF NOT EXISTS FOR (a:Allergy) ON (a.code);
    CREATE INDEX IF NOT EXISTS FOR (a:Address) ON (a.address);
    CREATE INDEX IF NOT EXISTS FOR (p:Procedure) ON (p.code);

    """
cypher_run(q)

<ipython-input-8-e79678bb85b2>:3: ExperimentalWarning: pandas support is experimental and might be changed or removed in future versions
  return result.to_df()


""


In [57]:

  # Create NEXT relationship between encounters of a patient
q = """
    CALL apoc.periodic.iterate(
    'MATCH (p:Patient) RETURN p',
    'MATCH (p)-[:HAS_ENCOUNTER]->(e)
    WITH e
    ORDER BY e.date
    WITH collect(e) AS encounters
    WITH encounters, encounters[1..] as nextEncounters
    UNWIND range(0,size(nextEncounters)-1,1) as index
    WITH encounters[index] as first, nextEncounters[index] as second
    CREATE (first)-[:NEXT]->(second)',{iterateList:false});
    
    """
cypher_run(q)

<ipython-input-8-e79678bb85b2>:3: ExperimentalWarning: pandas support is experimental and might be changed or removed in future versions
  return result.to_df()


,batches,total,timeTaken,committedOperations,failedOperations,failedBatches,retries,errorMessages,batch,operations,wasTerminated,failedParams,updateStatistics
0,1,122,0,122,0,0,0,{},"{'total': 1, 'committed': 1, 'failed': 0, 'err...","{'total': 122, 'committed': 122, 'failed': 0, ...",False,{},"{'nodesDeleted': 0, 'labelsAdded': 0, 'relatio..."


In [58]:
  # Pre-compute patient counts for each Condition
q = """
    CALL apoc.periodic.iterate('MATCH (c:Condition) RETURN c',
      'MATCH (c)<-[:HAS_CONDITION]-(:Encounter)<-[:HAS_ENCOUNTER]-(p:Patient)
      WITH c,count(p) AS NUM
      SET c.num=NUM', {}
      );
        """
cypher_run(q)

<ipython-input-8-e79678bb85b2>:3: ExperimentalWarning: pandas support is experimental and might be changed or removed in future versions
  return result.to_df()


,batches,total,timeTaken,committedOperations,failedOperations,failedBatches,retries,errorMessages,batch,operations,wasTerminated,failedParams,updateStatistics
0,1,185,0,185,0,0,0,{},"{'total': 1, 'committed': 1, 'failed': 0, 'err...","{'total': 185, 'committed': 185, 'failed': 0, ...",False,{},"{'nodesDeleted': 0, 'labelsAdded': 0, 'relatio..."


# Procedures Performed throught the Patient Journey
Where the condition ceased to exsist

In [73]:
q = """


MATCH (c)<-[:HAS_CONDITION]-(e)-[:HAS_END]->(e2),(e)<-[:HAS_ENCOUNTER]-(patient)
WITH patient, c, e, e2 LIMIT 10
MATCH (patient)-[r:HAS_ENCOUNTER]->(encounter)
WHERE e.date <= encounter.date <= e2.date
WITH patient, c, encounter ORDER BY encounter.date
WITH patient, c, collect(encounter) AS nodes
UNWIND nodes AS tE
MATCH (tE)-[:HAS_PROCEDURE]->(procedure)
RETURN patient.firstName AS firstName, patient.lastName AS lastName,
      c.code AS condition, collect(procedure.code) AS procedures

"""
df = cypher_run(q)
df

for p in df['procedures']:
    print(p)

['171207006', '715252007', '715252007', '171207006', '428211000124100', '386516004', '868187001', '715252007', '171207006', '710841007', '710824005', '430193006']


<ipython-input-8-e79678bb85b2>:3: ExperimentalWarning: pandas support is experimental and might be changed or removed in future versions
  return result.to_df()
